### CUDA Runtime error(2): Out of Memory

- Don't accumulate history across your training loop
- Don't hold onto tensors and variables you don't need
- Don't run RNNs on sequences that are too large
- Don't use linear layers that are too large

In [ ]:
total_loss = 0
for i in range(10000):
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output)
    loss.backward()
    optimizer.step()
    total_loss += loss
# 因为loss是一个需要求梯度的tensor，total_loss在整个循环的过程中会持续记录历史，占用显存，应该将loss里面的数据取出来
# 如果是一个实数值，可以用.item；如果是tensor，可以用　.data

In [ ]:
for i in range(5):
    intermediate = f(input[i])
    result += g(intermediate)
output  = h(result)
return output
# 在　h 函数退出之后，intermediate依然存活且占用显存，因此需要将不用的中间变量及时删除
# del intermediate

### My GPU memory isn't freed properly

PyTorch使用**caching memory allocator** 来加速显存分配，因此，`nvidia-smi`不能如实反映GPU显存使用情况。如果在PyTorch退出之后GPU显存依然没有释放，可能是因为某些Python的子进程还在运行，可以通过`ps -elf | grep python`找到这些进程病使用`kill -9 [pid]`杀掉进程。

### My recurrent network dosen't work with data parallelise

未完待续。。。

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class MyModule(nn.Module):
    # ... __init__, other methods
    def forward(self,padder_input,input_lengths):
        total_length = padded_input.size(1)
        packed_input = pack_padded_sequence(padded_input, input_lengths,batch_first = True)
        packed_output,_ = self.my_lstm(packed_input)
        output,_=pad_packed_sequence(packed_output,batch_first = True, total_length = total_length)
        return output

m = MyModule().data()
dp_m = nn.DataParallel(m)